In [1]:
%reload_ext autoreload
%autoreload 2
import autohf as hf
import numpy as np
from matplotlib import pyplot as plt
import time
import jax
import jax.numpy as jnp

In [2]:
symbols = ['H', 'H'] # Molecule
R = [np.array([0.0, 0.0, -0.694347576012433]), np.array([0.0, 0.0, 0.694347576012433])] # Optimal coordinates
num_elecs = 2 # Number of electrons

In [3]:
M1, M2 = hf.generate_basis_set('sto-3g', ['H', 'H']) # Generates default information for hydrogen
AO = []

# Generates atomic orbitals using the default information
for i, func in enumerate(M1 + M2):
    L, exp, coeff = func
    print(exp)
    AO.append(hf.AtomicBasisFunction(L, C=coeff, R=R[i]))

[3.425250914, 0.6239137298, 0.168855404]
[3.425250914, 0.6239137298, 0.168855404]


In [4]:
one_elec = hf.one_electron_integral_idx(num_elecs, AO, [0, 0])

In [5]:
jax.jacobian(one_elec)

<function jax._src.api.jacrev.<locals>.jacfun(*args, **kwargs)>

In [6]:
fn = jax.jit(lambda x : hf.atomic_norm((0, 0, 0), jnp.array([x, 1.0, 1.0]), jnp.array([1.0, 1.0, 1.0])))

In [7]:
fn(1.0)

DeviceArray(0.23756848, dtype=float32)

In [8]:
fn = jax.jit(lambda x : hf.electron_repulsion(jnp.array([1.0, 1.0, 1.0]), (0, 0, 0), 
                                 jnp.array([1.0, 1.0, 1.0]), jnp.array([1.0, 1.0, 1.0]), (0, 0, 0), 
                                 jnp.array([-1.0, 1.0, 1.0]), jnp.array([1.0, 1.0, 1.0]), (0, 0, 0), 
                                 jnp.array([1.0, -1.0, 1.0]), jnp.array([1.0, 1.0, 1.0]), (0, 0, 0), jnp.array([x, 1.0, -1.0])))

In [13]:
jax.lax.cond(True, lambda _: jax.lax.cond(True, lambda _: 3.0, lambda _: 2.0, operand=None), lambda _: 0.0, operand=None)

DeviceArray(3., dtype=float32)

In [31]:
def expansion_coeff(i, j, t, Ra, Rb, alpha, beta):
    """
    Computes expansion coefficients for calculation of overlap integrals
    """

    Qx = Ra - Rb
    p = alpha + beta
    q = (alpha * beta) / p

    return jax.lax.cond(
        jax.lax.bitwise_or((t < 0), (t > (i + j))),
        lambda _: 0.0,
        lambda _: jax.lax.cond(
            jax.lax.bitwise_and(jax.lax.bitwise_and(i == 0, j == 0), t == 0),
            lambda _: jnp.exp(-1 * q * (Qx ** 2)),
            lambda _: jax.lax.cond(
                j == 0,
                lambda _: (1 / (2 * p)) * expansion_coeff(i - 1, j, t - 1, Ra, Rb, alpha, beta) - \
                (q * Qx / alpha) * expansion_coeff(i - 1, j, t, Ra, Rb, alpha, beta) + \
                (t + 1) * expansion_coeff(i - 1, j, t + 1, Ra, Rb, alpha, beta),
                lambda _: (1 / (2 * p)) * expansion_coeff(i, j - 1, t - 1, Ra, Rb, alpha, beta) + \
                (q * Qx / beta) * expansion_coeff(i, j - 1, t, Ra, Rb, alpha, beta) + \
                (t + 1) * expansion_coeff(i, j - 1, t + 1, Ra, Rb, alpha, beta),
                operand=None
            ),
            operand=None
        ),
        operand=None
    )

In [32]:
expansion_coeff(2, 2, -2, 1.0, 0.0, 1.0, 1.0)

RecursionError: maximum recursion depth exceeded while calling a Python object